<div>
    <h1 style="margin-top: 50px; font-size: 33px; text-align: center">Homework 3 - Find the perfect place to stay in Texas!</h1>
    <br>
    <div style="font-weight:200; font-size: 20px; padding-bottom: 15px; width: 100%; text-align: center;">
        <right>Davide Toma, Fabio Montello, Umawhawhire Bonaventure</right>
        <br>
    </div>
    <hr>
</div>


The homework consists in analyzing the text of Airbnb property listings and building a search engine.

## Step 0: Preparation code

In this part we will write some functions to make sure all the libraries that will be needed along the run of the program are available. We divided this in three main block of code: a function to install missing libraries, a block to check that the libraries that usually needs to be installed are already in the system and a general import of the other libraries that are for sure part of Python packages suite.

In the first part we define a function that will let us install a package in the future, if it does not exist in the local packages. The library will be installed through `pip` just if it does actually exists in the repositories.

In [1]:
def install(package):
    try:
        import subprocess
        import sys
        subprocess.call([sys.executable, "-m", "pip", "install", package])
        print("Library installed")
    except:
        print("Cannot install the library")

In the second part we try to install all the libraries that usually are not part of the local set of packages. In the case the library is already available, we import it, otherwise we install it with the previous function, and then try again to import it.

In [2]:
try:
    from langdetect import detect
except:
    print("The library LanfDetect is not in the repository, I will install it")
    install('langdetect')
    from langdetect import detect

try:
    from texttable import Texttable
except:
    print("The library TextTable is not in the repository, I will install it")
    install('texttable')
    from texttable import Texttable

try:
    from geopy.geocoders import Nominatim
    import geopy.distance
except:
    print("The library geopy is not in the repository, I will install it")
    install('geopy')
    from geopy.geocoders import Nominatim
    import geopy.distance

try:
    import tabulate
except:
    print("The library tabulate is not in the repository, I will install it")
    install('tabulate')
    import tabulate

In the last block we import the common libraries that usually comes with Python (supposing that Anaconda is installed as well). In particular we import `unicodecsv` to not have problems with the import and export of text with special caracters. `Pickle` will be used to import/export the dictionaries into files, `nltk` is used to clean the sentences from the stopwords and for stemming, `langdetect` to understand in which language a text is written. `Pandas` is used to load the initial CSV and do some operations in a quicker way.
After all the imports, we download/update the list of the stopwords with the command `nltk.download('stopwords')`. `folium` is to create a map displaying the points and `re` is to apply regex to a string.

In [3]:
import pandas as pd
import unicodecsv as csv # We also import the unicodecsv library to handle any Unicode objects in the CSV report...as csv.
import nltk
import string
import pickle
import folium
import re
import json

from time import sleep
from IPython.display import HTML, display
from nltk import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from heapq import heappush, heappop
from scipy import spatial

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Step 1: Load the Data

Now we want to read the data from the initial CSV. We start by defining the directory of the data and then running the command to read the CSV.

In [4]:
DIR = 'data/'

df = pd.read_csv(DIR+"Airbnb_Texas_Rentals.csv", sep=",") # we read the dataset


We want then to clean the data we will not be using. In particular the following actions has been executed:
- Remove the column `Unamed: 0` which is useless for our analysis
- Remove all the rows which do not have a description, a latitude or a longitude
- Remove all the rows where the description is a `\` or a `.`
- Replace the new lines characters with a space in the whole dataframe
- Define a new column language, with undefined language for each row (will be updated later)

Finally we display the table, useful later for reference

In [5]:
df = df.drop('Unnamed: 0',axis = 1) #this column is useless for our analisys
df = df[pd.notnull((df['description'])) & pd.notnull((df['latitude'])) & pd.notnull((df['longitude'])) & (df['description'] != "\\" ) & (df['description'] != "." )] #If the row has no description, for us is useless, so we drop it
df = df.replace(r'\\n',' ', regex=True) #Remove all the new lines in the dataframe
df['language'] = 'nd'
df.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,nd
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,nd
2,$59,1,Houston,January 2017,'River house on island close to the city' A w...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,nd
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,nd
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,nd


We also retrieve the list of all the locations we need, and assign to them an id. These will be used later on in the software:

In [6]:
locationsList = {elem.lower() : i for i, elem in enumerate(set(df["city"]))}
locationsList

{'alamo heights': 0,
 'bellaire': 1,
 'tuscola': 2,
 'groesbeck': 3,
 'dayton': 4,
 'nemo': 5,
 'weatherford': 6,
 'kerens': 7,
 'caldwell': 8,
 'arlington': 9,
 'clifton': 10,
 'decatur': 11,
 'deer park': 12,
 'farmersville': 13,
 'silsbee': 14,
 'ingram': 15,
 'richland hills': 16,
 'fort bend county': 17,
 'benbrook': 18,
 'hereford': 19,
 'sunset': 20,
 'alvarado': 21,
 'celina': 22,
 'comfort': 23,
 'burnet': 24,
 'montgomery county': 25,
 'jasper': 26,
 'new waverly': 27,
 'alvin': 28,
 'mertens': 29,
 'fort worth': 377,
 'larue': 31,
 'jacksboro': 32,
 'quitman': 33,
 'burleson': 34,
 'glen rose': 35,
 'granbury': 36,
 'kemp': 37,
 'keller': 130,
 'tiki island': 39,
 'kyle': 453,
 'lancaster': 41,
 'whitewright': 42,
 'corsicana': 43,
 'canyon': 44,
 'clear lake shores': 45,
 'copper canyon': 46,
 'hudson oaks': 47,
 'bolivar peninsula': 48,
 'pearland': 49,
 'crystal beach': 57,
 'lago vista': 51,
 'willis': 52,
 'gatesville': 53,
 'round rock': 54,
 'kingsland': 55,
 'frederi

## Step 2: Create documents

Now from every row of the table, we want to create a `.tsv` file which will store all the relevant informations about a single istance that will be displayed later on in the search engine.

In [7]:
for row in df.iterrows():
    file_name = DIR + 'doc_'+str(row[0])+'.tsv' # We build dinamically the path of every file
    
    with open(file_name,'wb') as file: # We create/open the file. Wb stands for binary mode.
        csv_writer = csv.writer(file, delimiter = '\t') # Now we are writing the files as tab separated files.
        csv_writer.writerow(list(row[1].values)) # writerow want a list object ad input, so we used list.
        file.close() # Once the file is written, we close it, so we free some memory


After creating all the documents we will work with, now some more data to retrieve for the point 4 of our exercise. We are in fact going to retrieve the coordinates for the center of every city where there is an airbnb, so we can then do further computation to get a rank based also on the position of the B&B in respect to the city center.
To do so, starting from the location list we got previously, we retrieve the center through `geopy` and we add it to a list, containing the city ID and the coordinates of the center.
Please notice, since the following block of code can be very slow, expecially with a low connection, we saved a copy of the data retrived and executed this block just once. So we **strongly suggest to skip downloading the data again** (skip running the following block), and instead be sure you got the file `citycoord.pkl` from the Github repository in the `data/docs` directory. We will upload that file later on in the program, when needed

In [52]:
citycoord = {}

for key in locationsList:
    geolocator = Nominatim(user_agent="search engine")
    for i in range(5): #
        try:
            location = geolocator.geocode(key)
            print(location)
            coords_2 = ((location.latitude, location.longitude))
            break
        except: 
            sleep(0.1)
            pass
    citycoord[locationsList[key]] = coords_2
    
with open(DIR+'citycoord.pkl', 'wb') as file:
        pickle.dump(citycoord, file, pickle.HIGHEST_PROTOCOL)

Alamo Heights, Bexar County, Texas, USA
Bellaire, Harris County, Texas, 77401, USA
Tuscola County, Michigan, USA
Groesbeck, Limestone County, Texas, 76642, USA
Dayton, Montgomery County, Ohio, USA
Nemo, Morgan County, Tennessee, USA
Weatherford, Parker County, Texas, 76086, USA
Kerens, Navarro County, Texas, 75144, USA
None
None
None
None
None
Arlington, Tarrant County, Texas, USA
Clifton, Bristol, City of Bristol, South West England, England, BS8 3JH, UK
Decatur, Macon County, Illinois, USA
Deer Park, Harris County, Texas, USA
Farmersville, Tulare County, California, 93223, USA
Silsbee, Hardin County, Texas, USA
Ingram, Mendocino County, California, USA
Richland Hills, Tarrant County, Texas, 76118, USA
Fort Bend County, Texas, USA
Benbrook, Tarrant County, Texas, 76126, USA
Hereford, Herefordshire, West Midlands, England, HR1 2NB, UK
Sunset, Sutter County, California, 95953, USA
Alvarado, Veracruz de Ignacio de la Llave, México
Cellina, Unione dei comuni del Medio Verbano, VA, LOM, 21

Forreston, Ogle County, Illinois, USA
Alpine County, California, USA
Stephenville, Erath County, Texas, USA
Firenze, FI, TOS, Italia
Pontotoc County, Mississippi, USA
Winnsboro, Fairfield County, South Carolina, USA
Corinth, Alcorn County, Mississippi, USA
Keene, Cheshire County, New Hampshire, USA
Cedar Hill, Dallas County, Texas, USA
Dale County, Alabama, USA
Van Alstyne, Grayson County, Texas, USA
Millican, Brazos County, Texas, 77866, USA
Dublin, County Dublin, Leinster, Ireland
Sargent County, North Dakota, USA
New Ulm, Brown County, Minnesota, USA
Trophy Club, Denton County, Texas, TX 76262, USA
Westworth Village, Tarrant County, Texas, USA
Hempstead, Waller County, Texas, 77445, USA
Blanco County, Texas, USA
Port Arthur, Jefferson County, Texas, USA
Martindale, Calgary, Alberta, T3J 0H6, Canada
Flint, Flintshire, Wales, CH6 5AJ, UK
Llano County, Texas, USA
Schertz, Guadalupe County, Texas, USA
Damon, Albemarle County, Virginia, 22969, USA
Lantana, Palm Beach County, Florida, USA

Anson County, North Carolina, USA
Krum, Denton County, Texas, USA
Fulshear, Fort Bend County, Texas, 77441, USA
Trinidad, Las Animas County, Colorado, USA
Canyon Lake, Riverside County, California, USA
Brenham, Washington County, Texas, USA
Venus, Tenares, Hermanas Mirabal, R.D.
Bandera, Bandera County, Texas, USA
Roanoke, Virginia, USA
Georgetown, Scott County, Kentucky, 40324, USA
Hawkins County, Tennessee, USA
Justin, Denton County, Texas, USA
Galveston County, Texas, USA
Bastrop, Bastrop County, Texas, 78602, USA
Kennedale, Tarrant County, Texas, 76060, USA
Lake Jackson, Brazoria County, Texas, 77566, USA
Hearne, Robertson County, Texas, 77859, USA
Breckenridge, Summit County, Colorado, 80424, USA
Baird, Callahan County, Texas, 79504, USA
Rio Frio, Arrondissement de Hinche, Département du Centre, Ayiti
Wimberley, Hays County, Texas, USA
Paige, Caroline County, Virginia, USA
Cedar Creek, Westmoreland County, Pennsylvania, 15492, USA
Katy, Fort Bend County, Texas, USA
Mineola, Nassau

## 3.1: Conjunctive query

We want now to be able to make a dictionary that, given a query, returns us a list of documents conatining all the words in that query. We start by creating a dictionary that will map for every document the single words that title and description will contain, cleaned of the stopwords and stemmed. 
Since there are a lot of different languages used for the descriptions (with different alphabets as well), we decided to keep just the 4 most used languages: English, German, Spanish and Dutch. All the other languages will not be considered.

In [8]:
docdict = {}

languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}

for i, row in df[:].iterrows(): # For every row of the dataframe, we get values and index

    #We merge title and description (since we will consider both) and remove punctuation
    text = (str(row['description'])+ ' '+ str(row['title'])).translate({key: None for key in string.punctuation})
    
    
    if(text): # If the text exists
        try: # Try to detect the language
            lan = detect(str(text))
            df.loc[i,'language'] = lan # Set the language in the row

            if (lan in languages): # If it's one of the languages considered
        
                # Remove all the stopwords and all the non alphabetic words
                tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
        
                # Stem all the words and add them to a dictionary
                docdict[i] = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
            else:
                pass
        except:
            pass

docdict

{0: ['welcom',
  'stay',
  'privat',
  'room',
  'queen',
  'bed',
  'detach',
  'privat',
  'bathroom',
  'second',
  'anoth',
  'privat',
  'bedroom',
  'sofa',
  'bed',
  'avail',
  'addit',
  'addit',
  'iah',
  'airport',
  'airport',
  'avail',
  'privat',
  'iah',
  'airport'],
 1: ['fulli',
  'remodel',
  'home',
  'upscal',
  'nw',
  'alamo',
  'height',
  'amaz',
  'locat',
  'hous',
  'conveni',
  'locat',
  'quiet',
  'beauti',
  'season',
  'prestigi',
  'neighborhood',
  'close',
  'loop',
  'featur',
  'open',
  'floor',
  'origin',
  'hardwood',
  'full',
  'bathroom',
  'independ',
  'room',
  'sleep',
  'european',
  'inspir',
  'kitchen',
  'driveway',
  'park',
  'uniqu',
  'alamo',
  'height',
  'design',
  'inspir'],
 2: ['hous',
  'island',
  'close',
  'well',
  'maintain',
  'river',
  'hous',
  'san',
  'jacinto',
  'river',
  'extra',
  'room',
  'temporari',
  'river',
  'hous',
  'near',
  'citi'],
 3: ['privat',
  'bedroom',
  'cute',
  'littl',
  'home',


Next we want to map once every single stemmed word with a corresponding sequential ID into a dictionary. This will improve the speed later on.

In [9]:
voc = {}
cont = 0

for key, value in docdict.items(): # for every document
    for elem in value: # for every word
        if(elem not in voc): # if we did not map it yet
            voc[elem] = cont # map it with a sequential value
            cont += 1 # and increase the value
            
voc

{'welcom': 0,
 'stay': 1,
 'privat': 2,
 'room': 3,
 'queen': 4,
 'bed': 5,
 'detach': 6,
 'bathroom': 7,
 'second': 8,
 'anoth': 9,
 'bedroom': 10,
 'sofa': 11,
 'avail': 12,
 'addit': 13,
 'iah': 14,
 'airport': 15,
 'fulli': 16,
 'remodel': 17,
 'home': 18,
 'upscal': 19,
 'nw': 20,
 'alamo': 21,
 'height': 22,
 'amaz': 23,
 'locat': 24,
 'hous': 25,
 'conveni': 26,
 'quiet': 27,
 'beauti': 28,
 'season': 29,
 'prestigi': 30,
 'neighborhood': 31,
 'close': 32,
 'loop': 33,
 'featur': 34,
 'open': 35,
 'floor': 36,
 'origin': 37,
 'hardwood': 38,
 'full': 39,
 'independ': 40,
 'sleep': 41,
 'european': 42,
 'inspir': 43,
 'kitchen': 44,
 'driveway': 45,
 'park': 46,
 'uniqu': 47,
 'design': 48,
 'island': 49,
 'well': 50,
 'maintain': 51,
 'river': 52,
 'san': 53,
 'jacinto': 54,
 'extra': 55,
 'temporari': 56,
 'near': 57,
 'citi': 58,
 'cute': 59,
 'littl': 60,
 'situat': 61,
 'covet': 62,
 'garden': 63,
 'acr': 64,
 'access': 65,
 'campus': 66,
 'recent': 67,
 'purchas': 68,
 'mil

Now we want to create an inverted index that will map every word (using the ID we created previously) to a list of all the documents that contain that single word. 
The result will be in the following format: `{1: [0,2,5], 245: [1, 4, 7]}`.

In [10]:
invidx = {} 

for key, value in docdict.items():
    for elem in value:
        if(voc[elem] in invidx):
            invidx[voc[elem]] = invidx[voc[elem]] + [key]
        else:
            invidx[voc[elem]] = [key]
invidx

{0: [0,
  4,
  22,
  50,
  58,
  58,
  70,
  70,
  150,
  196,
  199,
  222,
  236,
  275,
  288,
  296,
  332,
  384,
  429,
  439,
  443,
  449,
  463,
  474,
  475,
  504,
  540,
  549,
  567,
  589,
  607,
  610,
  670,
  672,
  708,
  735,
  769,
  771,
  793,
  812,
  815,
  816,
  826,
  851,
  855,
  862,
  896,
  926,
  944,
  976,
  1024,
  1052,
  1058,
  1099,
  1138,
  1175,
  1187,
  1238,
  1240,
  1254,
  1257,
  1259,
  1305,
  1354,
  1366,
  1391,
  1391,
  1397,
  1411,
  1506,
  1540,
  1542,
  1552,
  1561,
  1581,
  1582,
  1589,
  1593,
  1605,
  1605,
  1611,
  1627,
  1646,
  1674,
  1695,
  1695,
  1701,
  1772,
  1839,
  1850,
  1902,
  1967,
  1969,
  1970,
 ...}

Now we define a search function that can be called every time and will gives us back a list of all the items that contains all the words requested in the query (apart from the stopwords). The function is splitted mainly in three parts: 
- Recognizing the language and stemming the query. If the query is not recognized as one of the languages written before, then we consider it and stem it as English.
- Getting the lost of documents containing all the words requested in the query
- Printing the results

In [11]:
def searchConj(string, n):
    
    text = string 
    lan = detect(str(text)) # Detect the language
    
    if(lan not in languages): 
        lan = 'en'
        
    # Remove non alphanumeric text and stem the words
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    # Get the list of documents
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx[voc[elem]]) # & does the union of the sets

    table = [['<b>Title</b>', '<b>Description</b>', '<b>City</b>', '<b>URL</b>']]
    
    # Open every document and add it to a list
    for elem in listofdocs:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            
            table = table + [[text[7], text[4], text[2], '<a href ="' + str(text[8]) + ' target= "_blank">' + str(text[8]) + '</a>' ]]
            
            file.close()

    # Display the list
    display(HTML(tabulate.tabulate(table[:n+1], tablefmt='html')))   
    

Now we call the search function, letting the user decide the query to input, and limiting the outputs to the first 5 elements.

In [12]:
searchConj(input(), 5)

House with garden


Title,Description,City,URL
"Rosen House Inn, Rose Garden Room","We are a small family owned Bed and Breakfast located in the Heart of the near Southside. Within walking distance to small shops, bars, and local dining. We offer snacks, coffee, tea, and sodas as well as a full hearty breakfast for our guests.",Fort Worth,https://www.airbnb.com/rooms/9972447?location=Benbrook%2C%20TX
"BRAND NEW HOME, EXQUISITE DECORATION, LARGE GARDEN","Brand new house with exquisite decoration. We have two indoor parking spaces and three exterior parking spaces in case you are traveling by car. The beds are orthopedic which provides a comfortable rest. For us cleaning is an essential point in the house, so feel totally safe. The house has been designed and decorated to satisfy the needs of families, business trips and relaxation time. We hope to make your stay an unforgettable experience!",Spring,https://www.airbnb.com/rooms/15285127?location=Conroe%2C%20TX
East Austin Hillside Gem,"Beautiful and modern 3Br, 2.5Ba located minutes from downtown. Amenities include front deck, back patio, peaceful backyard with tiered gardens and fire pits. There is also gym equipment, wifi, Xbox, and cable with HBO, Showtime, Starz, etc. There is free offstreet parking for two cars. Perfect location for families(with kids or furry friends,) couples, business travelers or friends looking for a spacious and comfortable location with plenty of amenities. This house has beautiful sunset views of downtown Austin just 6 miles from Darrell K Royal Stadium, 8 miles from Zilker Park and 14 miles from Circuit of the Americas.",Austin,https://www.airbnb.com/rooms/17555039?location=Bastrop%20County%2C%20TX
Charming 1 BR cottage near Chappell Hill,"Welcome to Cedar Meadows, a one-of-a-kind 2015 custom-built cottage situated on 7-acres of land with gardens all around. Surrounded by a white picket fence with decks on either side of the house, this property will allow you to enjoy the stunning views in any weather. The house is located on North Meyersville Road, 4.5 miles from Chappell Hill and 9.5 miles from Brenham making it convenient to access either town but giving you a feeling of secluded haven.",Brenham,https://www.airbnb.com/rooms/17921211?location=Brenham%2C%20TX
Gibbs House / Walk to the Beach,"Your private suite is in my personal residence. I live here. It takes 2 minutes to walk to the Convention Center and the beach is a three minute walk away. A theme park (The Pleasure Pier) is two miles away on the Galveston Seawall. Moody Gardens is about four miles away. A Kroger grocery store (with a Starbucks Coffee Shop) is a three minute walk from the house. Gibbs House is excellent for couples, business travelers, small families, and solo adventurers.",Galveston,https://www.airbnb.com/rooms/16982846?location=Bayou%20Vista%2C%20TX


In [13]:
searchConj(input(), 5)

Increíble habitación lugares


Title,Description,City,URL
Cuarto luminoso,"Lugares de inter\xc3\xa9s: transporte p\xc3\xbablico, parques. Te va a encantar mi espacio porque Hay mucho verde y espacios con juegos para ni\xc3\xb1os y chanchas de futbol. Parrilla y mesas! , La ubicaci\xc3\xb3n es muy buena, es una zona muy linda con muchos restorantes y supermercados, farmacias cercas., el ambiente Es agradable.. Mi alojamiento es bueno para parejas, aventureros, viajeros de negocios y mascotas.",Irving,https://www.airbnb.com/rooms/16073393?location=Coppell%2C%20TX
"House for Rent at Brownsville, Tx.","Lugares de inter\xc3\xa9s: VICC Country Golf Club, Schlitterbahn Resort Waterpark, Mercedes Outlet, Sunrise Mall, Shopping, South Padre Island, ValleyTennis Center. Te va a encantar mi espacio por Comodidad, ubicaci\xc3\xb3n, acogedor, practico. Mi alojamiento es bueno para familias (con hijos) y grupos grandes.",Brownsville,https://www.airbnb.com/rooms/16649395?location=Brownsville%2C%20TX
Peaceful home 25 min from Airport &35 min Downtown,"Lugares de inter\xc3\xa9s: el aeropuerto se encuentra a 25 minutos de distancia, el centro de la ciudad se encuentra a 35 min, en el centro hay mucha vida nocturna , transporte p\xc3\xbablico a 10 minutos de la casa hay autobuses que van a el centro . Te va a encantar mi lugar debido a los espacios, y la casa tiene un ambiente muy acogedor. la tranquilidad, los techos altos. Mi alojamiento es bueno para viajeros de negocios o turismo.",San Antonio,https://www.airbnb.com/rooms/15773671?location=Cibolo%2C%20TX
Houston 15 min NRG,Lugares de inter\xc3\xa9s: Hobby airport. Te va a encantar mi espacio por Extremely relax. Mi alojamiento es bueno para aventureros y familias (con hijos).,Houston,https://www.airbnb.com/rooms/16973563?location=Channelview%2C%20TX
Peaceful home 25 min from Airport &35 min Downtown,"Lugares de inter\xc3\xa9s: el aeropuerto se encuentra a 25 minutos de distancia, el centro de la ciudad se encuentra a 35 min, en el centro hay mucha vida nocturna , transporte p\xc3\xbablico a 10 minutos de la casa hay autobuses que van a el centro . Te va a encantar mi lugar debido a los espacios, y la casa tiene un ambiente muy acogedor. la tranquilidad, los techos altos. Mi alojamiento es bueno para viajeros de negocios o turismo.",San Antonio,https://www.airbnb.com/rooms/15773671?location=Bulverde%2C%20TX


## 3.2: Conjunctive query & Ranking score

Now we have to create a second search engine that, given a query, will return the top-k documents ordered by the similarity (cosine similarity).
First of all we create the tf_idf function that will compute the tfidf (term frequency–inverse document frequency), given some parameters as input: 
- `word` the word we are interested in calculating the frequency 
- `doc` the text we want to evaluate the frequency of the word
- `D`    the total number of documents (a constant value)
- `N`    the number of documents containig that word.

In [14]:
#function tfidf
def tf_idf(word,doc,D,N):
    import math
    f = doc.count(word) / len(doc)
    tfidf = math.log(D/N)*f
    
    return (tfidf)   

After defining the key function for the ranking score, we have to create a second inverted index. We start from duplicating the previous index and then taking into account for every word in which document it appears, without considering the duplicates. At the end we count how many documents we have in total.

In [15]:
invidx2 = invidx.copy()
alldocs = []
for key, value in invidx2.items():
    alldocs = alldocs + invidx2[key]
    invidx2[key] = set(value)

Ndocs = len(set(alldocs))
print(Ndocs)

18122


The next step is to calculate the actual *tf_idf* value from the previous index using the function we declared two code blocks above. So for every document in every word, we create a pair *(number of document, tf_idf)*, and we put all these values into a new index.

In [17]:
inv_voc = {v: k for k, v in voc.items()}
invidx3 = invidx2.copy()

for key, value in invidx2.items():
    tmp = []
    for elem in value:
        x = tf_idf(inv_voc[key], docdict[elem], (Ndocs + 1), len(value))
        tmp.append((elem,x))
    invidx3[key] = tmp

As an example, we visualize the array with all the pairs for the word *stay*:

In [18]:
print(invidx3[1])

[(0, 0.0950879315309583), (8192, 0.09904992867808156), (8204, 0.18286140679030444), (13, 0.05798044605546238), (16401, 0.12511569938283987), (16403, 0.09143070339515222), (8215, 0.06603328578538771), (8218, 0.050578686984552285), (30, 0.055283681122650176), (16417, 0.055283681122650176), (39, 0.10115737396910457), (46, 0.06791995109354164), (8240, 0.11319991848923606), (8245, 0.07668381575077282), (8246, 0.05282662862831017), (57, 0.055283681122650176), (8266, 0.054027233824408126), (16462, 0.06791995109354164), (16464, 0.10335644731625902), (16466, 0.18286140679030444), (85, 0.05942995720684894), (8277, 0.13206657157077542), (16475, 0.1584798858849305), (8284, 0.12511569938283987), (100, 0.1584798858849305), (8294, 0.036855787415100115), (8296, 0.06791995109354164), (109, 0.05659995924461803), (8301, 0.05798044605546238), (8303, 0.06424860238578264), (16494, 0.09904992867808156), (8315, 0.07428744650856117), (8319, 0.06424860238578264), (8325, 0.08804438104718361), (8326, 0.0990499286

Once the index has been created, we save the variable in a file, so we wil be able to reuse it very quickly every time we will use the search engine, without the need to recompile the *tf_idf* every time.

In [19]:
with open(DIR+'dicIndex.pkl', 'wb') as file:
        pickle.dump(invidx3, file, pickle.HIGHEST_PROTOCOL)

We define now a function that let us order a dictionary using an heap. This will be used later.

In [20]:
def heapsort(d):
    l = []
    for key in d:
        heappush(l, (d[key], key))
    return [heappop(l) for i in range(len(l))]

Now we want to find the documents most similar in according to the query the user give in input and we want as output of this second search engine, the first k-documents and in  particular:

- Title
- Description
- City
- Url
- Similarity

For this porpouse we build a function that let us search starting from a given query and the number of results that we want back. The function is basically split in three parts:

- Transforming the query and calculating it's *tf_idf*
- Calculate the cosine similarity between the query and the documents
- Print the output

Before comparing the query with the documents, we want to *standardize* it as we did previously, by recognizing the language, removing numbers and puntaction and stemming the words. If the language is not one of the defaults we defined previously (English, Dutch, German or Spanish), we will search for English text.
For the cosine similarity we compute it with the function `spatial.distance.cosine` between the two vectors (the one with the tfidf of the query and the other one with the tfidf of the words we can fine in the documents where we have all the words of the query).
Finally we display the result printing it in HTML, using the `table` tags implemented in the tabulate function.

In [21]:
def searchCos(string, n):
    
    # Open the index variable we created previously
    with open(DIR+'dicIndex.pkl', 'rb') as file:
        data = pickle.load(file)
        
    # Standardize the query + tf_idf
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    query_tfidf = [tf_idf(i,text,(Ndocs + 1),1) for i in result]
    
    # Extracting just the words that appears from the query
    finaldic = {}
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            for k in invidx3[voc[elem]]:
                if(k[0] in finaldic):
                    finaldic[k[0]] = finaldic[k[0]] + [k[1]]
                else:
                    finaldic[k[0]] = [k[1]]

    final = {} 
    
    # Extracting the documents that uses does words and calculating the cosine similarity
    for k in finaldic:
        if(len(finaldic[k]) == len(result)):
            final[k] = finaldic[k]
    for k in final:
        final[k] = 1 - spatial.distance.cosine(final[k] , query_tfidf) 
            
    # Sorting the result
    final = heapsort(final)
    
    # Printing the result as HTML
    table = [['<b>Doc</b>','<b>Title</b>', '<b>Description</b>', '<b>City</b>', '<b>URL</b>', '<b>Similarity</b>']]
    for elem in final[:n:-1]:
        file_name = DIR + 'doc_'+str(elem[1])+'.tsv'
        with open(file_name,'rb') as file: 
            text = str(file.read()).split('\\t')
            table = table + [[str(elem[1]), text[7], text[4], text[2], '<a href ="' + str(text[8]) + ' target= "_blank">' + str(text[8]) + '</a>', elem[0]]]
            file.close()
    display(HTML(tabulate.tabulate(table[:n+1], tablefmt='html')))


In [22]:
searchCos(input(), 5)

House with garden


Doc,Title,Description,City,URL,Similarity
14254,Garden Bedroom,"Comfortable and private, the garden bedroom is ideal for a long weekend or festival stay. It features a private closet, a full-sized bed and a lovely view of the backyard. Common spaces are here for your use as well. Cook a meal and stick to your travel budget, sit in the living room front window with a cup of coffee and watch the peacocks in the morning or enjoy the sunshine and garden from the back patio. If being out and about in Austin is your goal, our neighborhood is central and there are many easy options for getting around the city. We have and love dogs. Please get in touch if you would like to bring yours! There are so many things to love about our neighborhood. Not only are we super close to great parkland, bars, restaurants, coffee shops, food trailers and vintage shops, we're surrounded by huge live oaks and peacocks! The neighbors are open and friendly and have made us feel welcome from the start. Some of our favorite places include Skylark Lounge, Contigo, Dai Due, Sugar Mama's, Coldtowne Theater, The Work Horse, Foreign and Domestic, Drink Well, Kome, Tyson's Tacos, Room Service Vintage, Uptown Modern, Michi Ramen, Big Bertha's and Lucy's Fried Chicken. We also really dig our backyard patio and grill, and we think you will, too. Our neighborhood connects directly to the Mueller Greenway trails, putting guests less than a mile away from The Thinkery and Mueller Park (they have a great Farmer's Market on Sundays and a beautiful loop for walkers and runners alike). By bike, take the Greenway two miles west to UT, North Loop or Hyde Park, take it a mile southeast to Cherrywood or Manor Road, five miles further and you're in the thick of the East Side & downtown without ever having to leave a bike lane. If cycling is not your thing, shared rides are abundant (getme is a good app) as are Car To Go cars. There is also a bus stop less than two blocks from the house, and we are two miles from the Capmetro station at Highland. I-35 is literally a stone's throw from the house, putting you just a few exits from wherever you want to be in Austin. If you are interested, we can provide bike rental services and route recommendations. We have two dogs, one big and one small. Although we are responsible owners and you will find the house clean, please consider this if you are afraid of dogs or have allergies. On the other hand, if you would like to bring your well-behaved dog, please reach out and we will see if this can be arranged. Our neighborhood connects directly to the Mueller Greenway trails, putting guests less than a mile away from The Thinkery and Mueller Park (they have a great Farmer's Market on Sundays and a beautiful loop for walkers and runners alike). By bike, take the Greenway two miles west to UT, North Loop or Hyde Park, take it a mile southeast to Cherrywood or Manor Road, five miles further and you're in the thick of the East Side & downtown without ever having to leave a bike lane. If cycling is not your thing, rideshares and cabs are abundant, as are Car To Go cars. There is also a bus stop less than two blocks from the house, and we are two miles from the Capmetro station at Highland. I-35 is a stone's throw from the house, putting you just a few exits from wherever you want to be in Austin. If you are interested, we can provide bike rental services and route recommendations.",Austin,https://www.airbnb.com/rooms/5096161?location=Austin%2C%20TX,0.990810173709318
9137,Greeley Gardens Carriage House,This is a beautiful cozy garage apartment with a large porch overlooking the grounds that make up the Greeley Gardens estate. The Apartment has low ceilings (less than 7ft) and is ideal for travelers looking for a serene indoor/outdoor environment to relax during their travels.,Houston,https://www.airbnb.com/rooms/8301898?location=Bellaire%2C%20TX,0.9796714437024007
6072,Private East End Garden Patio Home,"This is a very colorful electic house 15 min

# Step 4: Define a new score!

Now it's our turn to think how documents should be classified according to the query.
In the process of thinking what can give value in between two houses listed, we thought that the distance from the city center can be something that may really make the difference. Another aspect that we want to value is the price, cause think users may be more interest in cheaper places. Of course this is just an assumption, both for the choices of evaluating the distance from the city center and the price. One may suggest that the costumer is not always interested in staying close to the city center, but the same thing is correct for the price (one may want to see the most expensive rooms available) and by extension for any other feauture in the dataset, for example the number of rooms, which for somebody the larger the better, meanwhile for other people one room it is enough. So, independently from the rank chosen from us, if we want really to let the user find what he/she is looking for, the best solution is combining the rank score with some filter. This thing needs to be kept in mind in actual implementations, but it is not what was required from the homework (actually the opposite, it was asked to avoid it and so we did).

The next block code retrieve the variable containing all the city centers we downloaded in *Step 1*:

In [53]:
with open(DIR+'citycoord.pkl', 'rb') as file:
        citycenters = pickle.load(file)
        print(citycenters)

{0: (29.4849531, -98.4658502), 1: (29.7057858, -95.4588299), 2: (43.4776781, -83.3951071), 3: (31.5243379, -96.5338693), 4: (39.7589478, -84.1916069), 5: (36.0670188, -84.6524411), 6: (32.7589648, -97.7970748), 7: (32.1309147, -96.2285021), 8: (32.1309147, -96.2285021), 9: (32.701939, -97.105624826688), 10: (51.4615884, -2.6202846), 11: (39.8454163, -88.9524151), 12: (29.7052284, -95.1238204), 13: (36.2977283, -119.2067767), 14: (30.3490978, -94.1779626), 15: (38.8901809, -123.1719494), 16: (32.8159623, -97.2280695), 17: (29.511218, -95.7807348), 18: (32.673188, -97.4605759), 19: (52.0553813, -2.7151735), 20: (39.2504459, -121.651914), 21: (18.82867105, -95.9238843132054), 22: (45.8698992, 8.6116762), 23: (29.967715, -98.9050337), 24: (30.7591332, -98.2244422), 25: (39.2052633, -89.5057126), 26: (39.0082423, -88.1552371), 27: (40.7642073, -86.1924949), 28: (29.4238472, -95.2441009), 29: (5.3297222, 162.97), 377: (32.753177, -97.3327459), 31: (51.2074112, 26.5208033), 32: (33.2184477, -

Now we define the function that will return the distance from the house coordinates and the city center.

In [54]:
def distanceFromCityCenter(lat, long, city):
    city = city.lower()
    return geopy.distance.distance([lat, long], citycenters[locationsList[city]]).km

In the next cell we will have a dictionary with the ranking of the houses computed in this way:

$$score = (1 - \frac{distance}{max distance} )*0,7 + (1 - \frac{price}{max price})*0,3$$

From our scoring function you can see that for us the most important variable is the distance from the city center that has the 0,7 weight on the final score, and the price comes after the distance in importance.

In [60]:
rankdict = {}
maxprice = 0
for i, row in df[:].iterrows():
    # Calculate the distance from the city center of every house
    try:
        dist = (distanceFromCityCenter(row['latitude'], row['longitude'], row['city']))
    except:
        dist = 1000
        
    try:
        price = float((str(row['average_rate_per_night']))[1:])
    except:
        price = 0

    if (maxprice < price):
        maxprice = price
    
    # Temporary dictionary with distance, price and document
    try:
        if(locationsList[row['city'].lower()] in rankdict):
            rankdict[locationsList[row['city'].lower()]] = rankdict[locationsList[row['city'].lower()]] + [[i, dist, price]]
        else:
            rankdict[locationsList[row['city'].lower()]] =  [[i, dist, price]]
    except:
        pass
        
# Final document, with the distance and the score
for k in rankdict:
    m = max([i[1] for i in rankdict[k]]) # Relative distance
    for i, val in enumerate(rankdict[k]):
        rankscore = (1-val[1]/m)*0.7 + (1-(val[2]/maxprice))*0.3
        rankdict[k][i] = [rankscore, val[0]]

print(rankdict)

{236: [[0.767878431221366, 0], [0.6990537138934911, 53], [0.9706880253646419, 55], [0.5309614649560274, 91], [0.7829149871743736, 134], [0.3817948961901436, 420], [0.3333474823030831, 441], [0.9575715750410275, 445], [0.3239553688484519, 457], [0.32334674061568475, 484], [0.8354120946824606, 699], [0.8215992109862562, 849], [0.5210201814414201, 1041], [0.6751568143335913, 1356], [0.6064063331903456, 1428], [0.5792383523614839, 1555], [0.8944498136674729, 1611], [0.7197823109122179, 1633], [0.7066729294214671, 1768], [0.4079597916281581, 2511], [0.7066729294214671, 2993], [0.4922138065558561, 3228], [0.5210201814414201, 3260], [0.44792687456947844, 3366], [0.4079597916281581, 3412], [0.5782353651711385, 3439], [0.4402845022749329, 3686], [0.317189798929803, 3753], [0.7717772345240592, 3821], [0.7215624330827028, 3849], [0.77583026266002, 3857], [0.3865062514758586, 3898], [0.6024199174696313, 3944], [0.9706880253646419, 4173], [0.9575715750410275, 4303], [0.985673219890261, 4393], [0.60

Finally, this is the last search engine with our ranking score, so we will give in input the location (city) and then we will find the top-k houses in this city according to our search engine.
Once again we declare another function that will sort our score with a heap.

In [61]:
def heapsortd(d):
    l = []
    for e in d:
        heappush(l, [e[0], e[1]])
    return [heappop(l) for i in range(len(l))]

def searchByLocation(loc, n):
    
    try:
        #Find the location and sort the houses by rank
        loc = loc.lower()
        todisplay = (heapsortd(rankdict[locationsList[loc]])[:n:-1])

        #Print the result
        table = [['<b>Doc</b>','<b>Title</b>', '<b>Description</b>', '<b>City</b>', '<b>URL</b>', '<b>Rank</b>']]        
        for elem in todisplay:
            file_name = DIR + 'doc_'+str(elem[1])+'.tsv'
            with open(file_name,'rb') as file: 
                text = str(file.read()).split('\\t')
                table = table + [[str(elem[1]), text[7], text[4], text[2], '<a href ="' + str(text[8]) + ' target= "_blank">' + str(text[8]) + '</a>', elem[0]]]
                file.close()
        display(HTML(tabulate.tabulate(table[:n+1], tablefmt='html')))
    
    except:
        print("All the locations available are displayed")

In [62]:
searchByLocation(input(), 10)

dallas


Doc,Title,Description,City,URL,Rank
17809,Upscale Modern Loft Downtown Dallas,"One of a kind spacious loft near the heart of Downtown Dallas (perfect location). Ideal for business or leisure travelers seeking an unforgettable experience. Striking the perfect accord between modern artistic and industrial, this open loft provides guests with a comfortable one of a kind stay. Features a beautiful common roof top with spectacular downtown views. King bed, Queen bed, Apple TV, and WiFi.",Dallas,https://www.airbnb.com/rooms/17733183?location=Balch%20Springs%2C%20TX,0.9921914306180317
6847,Upscale Modern Loft Downtown Dallas,"One of a kind spacious loft near the heart of Downtown Dallas (perfect location). Ideal for business or leisure travelers seeking an unforgettable experience. Striking the perfect accord between modern artistic and industrial, this open loft provides guests with a comfortable one of a kind stay. Features a beautiful common roof top with spectacular downtown views. King bed, Queen bed, Apple TV, and WiFi.",Dallas,https://www.airbnb.com/rooms/17733183?location=Cedar%20Hill%2C%20TX,0.9921914306180317
13374,Sunrise Lounge,"Great lounge-style loft on the edge of downtown. Close to the DART. Sunrise suite with great separation of space, close to everything you need in Dallas.",Dallas,https://www.airbnb.com/rooms/18208527?location=Balch%20Springs%2C%20TX,0.9920052915496371
7510,Sunrise Lounge,"Great lounge-style loft on the edge of downtown. Close to the DART. Sunrise suite with great separation of space, close to everything you need in Dallas.",Dallas,https://www.airbnb.com/rooms/18208527?location=Cedar%20Hill%2C%20TX,0.9920052915496371
14260,Luxury High Rise In Downtown Dallas,"My place is good for couples, solo adventurers, and business travelers.",Dallas,https://www.airbnb.com/rooms/17513359?location=Arlington%2C%20TX,0.9862958700495585
1495,Luxury High Rise In Downtown Dallas,"My place is good for couples, solo adventurers, and business travelers.",Dallas,https://www.airbnb.com/rooms/17513359?location=Balch%20Springs%2C%20TX,0.9862958700495585
16245,Cosmopolitan Comfort Downtown,"Enjoy your private bedroom and bath in my roomy 2br sanctuary in the heart of Downtown. Very convenient to Convention Center, Deep Ellum, Uptown, Farmer's Market, the Museum and Theatre District, restaurants, and hotels and conference headquarters.",Dallas,https://www.airbnb.com/rooms/9460165?location=Balch%20Springs%2C%20TX,0.98433917557793
14970,Cosmopolitan Comfort Downtown,"Enjoy your private bedroom and bath in my roomy 2br sanctuary in the heart of Downtown. Very convenient to Convention Center, Deep Ellum, Uptown, Farmer's Market, the Museum and Theatre District, restaurants, and hotels and conference headquarters.",Dallas,https://www.airbnb.com/rooms/9460165?location=Arlington%2C%20TX,0.98433917557793
1750,Clean Downtown High-rise Condo Great Location!!!,"You can relax and enjoy a big-city lifestyle in an exclusive unique community. Beautifully appointed lofts overlooking Downtown Dallas. Just a quick walk to Uptown, outdoor fun at Klyde Warren Park, exploring your artistic side in the Dallas Arts District, or shopping fresh produce at the Farmers Market. Five-star dining, shopping, live music and entertainment all just minutes away. You're right next to the terrace and pool! CVS, restaurants, bars, bank and dry cleaners all inside the building!",Dallas,https://www.airbnb.com/rooms/14673897?location=Balch%20Springs%2C%20TX,0.9835759605890453
14625,Private Room in Downtown Townhouse,"Comfortable and private guest room with a queen size bed, desk, couch, TV, attached bathroom and more, located all by itself on the first floor of a 3 story downtown townhome. Enjoy the privacy of staying on your own floor, coming and going as you please using keypad locks and your own personal pass-code while at the same time enjoying all the comforts of home including a kitchen, living/dining area, covered parking, laundry and a rooftop terrace with 

## Bonus: Make a nice visualization!

Now it's time to make some maps.

We will ask in input to the user the coordinates and a radius and our goal is returning each house in our database that is inside the circle with center the coordinates given in input and the radius.
But for us is important also to visualize the price ranges of the houses we can find in the circle, so we color the markers of each house in the circle in the following way:

- Green: houses with a rate per night less than 100\$.
- Blue: houses with a rate per night more or equal than 100\$ and less than 600\$.
- Red: houses with a rate per night higher than 600\$.

in the marker you can also find the link to reach the announcement of the house you want to book.

In [6]:
lat = float(input())
lon = float(input())
c = [lat, lon]
r = int(input('please insert the radius in km: '))

29.4849531
-98.4658502
please insert the radius in km: 15


In [7]:
m = folium.Map(location = c, zoom_start = 9)

In [8]:
folium.Circle(
    radius = (r * 1000),
    location = c,
    popup = 'The Waterfront',
    color = 'crimson',
    fill = True,
).add_to(m)

In [9]:
m

In [10]:
Coords = {}
for doc in range(len(df)):
    f = open(DIR+'doc_'+str(doc)+'.tsv', 'r', encoding = 'utf-8')
    for line in f:
        ou = line.strip().split('\t')
        break
    if ou[8] not in Coords:
        if ou[6] != 'nan' and ou[7] != 'nan':
            Coords[ou[8]] = [float(ou[5]), float(ou[6]), int(ou[0].replace('$', ''))]

In [11]:
from geopy import distance

def google_maps(lat, lon):
    for k, v in Coords.items():
        P1 = c
        lat1 = v[0]
        lon1 = v[1]
        P2 = (lat1, lon1) 
        if (distance.distance(P1, P2).km) < r:
            if v[2] < 100 :
                tooltip = 'Click me!'
                folium.Marker(location = [float(P2[0]),float(P2[1])], 
                              popup = folium.Popup('<div><a href="'+k+'" target = "_blank" >'+k+'</a></div>'),
                              icon = folium.Icon(color = 'green', icon = 'home')
                             ).add_to(m)
            elif v[2]>= 100 and v[2] < 600:
                    tooltip = 'Click me!'
                    folium.Marker(location = [float(P2[0]),float(P2[1])], 
                                  popup = folium.Popup('<div><a href="'+k+'" target = "_blank" >'+k+'</a></div>'),
                                  icon = folium.Icon(color = 'blue', icon = 'home')
                                 ).add_to(m)
            elif v[2] >= 600:
                
                tooltip = 'Click me!'
                folium.Marker(location = [float(P2[0]),float(P2[1])], 
                                popup = folium.Popup('<div><a href="'+k+'" target = "_blank" >'+k+'</a></div>'),
                                icon = folium.Icon(color = 'red', icon = 'home')
                                ).add_to(m)
                
            
        else:
            pass

In [12]:
google_maps(c[0], c[1])

In [13]:
m.save('map.html')

m